# 原网址
https://mubu.com/doc/6IVyPGFT0UY

In [24]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/glm-4-9b-chat",revision = 'master',cache_dir='./model')
model_dir = snapshot_download("ZhipuAI/glm-4-9b-chat-1m",revision = 'master',cache_dir='./model')
model_dir = snapshot_download("ZhipuAI/glm-4v-9b",revision = 'master',cache_dir='./model')

2024-08-30 17:16:08,135 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2024-08-30 17:16:08,918 - modelscope - WARNING - Using branch: master as version is unstable, use with caution

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]
Downloading: 100%|██████████| 1.41k/1.41k [00:00<00:00, 2.39kB/s]

Downloading:   0%|          | 0.00/36.0 [00:00<?, ?B/s]
Downloading: 100%|██████████| 36.0/36.0 [00:00<00:00, 47.3B/s]

Downloading:   0%|          | 0.00/2.21k [00:00<?, ?B/s]
Downloading: 100%|██████████| 2.21k/2.21k [00:00<00:00, 3.40kB/s]

Downloading:   0%|          | 0.00/208 [00:00<?, ?B/s]
Downloading: 100%|██████████| 208/208 [00:00<00:00, 295B/s]

Downloading:   0%|          | 0.00/6.36k [00:00<?, ?B/s]
Downloading: 100%|██████████| 6.36k/6.36k [00:00<00:00, 10.4kB/s]

Downloading:   0%|          | 0.00/1.82G [00:00<?, ?B/s]
Downloading:   0%|          | 1.00M/1.82G [00:00<18:10, 1.79MB/s]
Downloading:   2%|▏         | 34.0M/1.82G [00:

# 文本推理

In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("model/ZhipuAI/glm-4-9b-chat", trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "model/ZhipuAI/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Downloading:  70%|██████▉   | 1.25G/1.80G [00:16<00:01, 449MB/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]


你好👋！很高兴见到你，有什么可以帮助你的吗？


# 多模态推理

In [38]:
!wget https://api2.mubu.com/v3/document_image/28777116_f2aa961a-0b94-4bf2-a16c-334dec8c6b39.png -O ./images/temp.png

--2024-08-30 17:30:21--  https://api2.mubu.com/v3/document_image/28777116_f2aa961a-0b94-4bf2-a16c-334dec8c6b39.png
正在解析主机 api2.mubu.com (api2.mubu.com)... 180.184.178.163
正在连接 api2.mubu.com (api2.mubu.com)|180.184.178.163|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 302 
位置：https://document-image.mubu.com/document_image/28777116_f2aa961a-0b94-4bf2-a16c-334dec8c6b39.png [跟随至新的 URL]
--2024-08-30 17:30:21--  https://document-image.mubu.com/document_image/28777116_f2aa961a-0b94-4bf2-a16c-334dec8c6b39.png
正在解析主机 document-image.mubu.com (document-image.mubu.com)... 61.153.154.96, 117.68.68.217, 240e:10:4801:213b:8000:0:b00:226, ...
正在连接 document-image.mubu.com (document-image.mubu.com)|61.153.154.96|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 1197421 (1.1M) [image/png]
正在保存至: ‘./images/temp.png’

./images/temp.png   100%[===================>]   1.14M  --.-KB/s    用时 0.09s   

2024-08-30 17:30:21 (12.3 MB/s) - 已保存 ‘./images/temp.png’ [1197421/1197421])



In [39]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("model/ZhipuAI/glm-4v-9b", trust_remote_code=True)

query = '描述这张图片'
image = Image.open("images/temp.png").convert('RGB')
inputs = tokenizer.apply_chat_template([{"role": "user", "image": image, "content": query}],
                                       add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                       return_dict=True)  # chat mode

inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "model/ZhipuAI/glm-4v-9b",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 214.00 MiB. GPU 

# webui推理

<ol>
<li>git clone https://github.com/THUDM/GLM-4.git</li>
<li>cd  GLM-4-main/composite_demo</li>
<li>
    <ul>
        <li>conda create -n glm-4-demo python=3.12</li>
        <li>conda activate glm-4-demo</li>
        <li>pip install -r requirements.txt</li>
    </ul>
<li>streamlit run src/main.py</li>
</ol>

In [46]:
!git clone https://github.com/THUDM/GLM-4.git
!pip install --upgrade pip
!pip install -r ./GLM-4/composite_demo/requirements.txt

fatal: 目标路径 'GLM-4' 已经存在，并且不是一个空目录。
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
    torch (>=1.9.*)
           ~~~~~~^
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 129.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 146.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 127.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 140.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 163.6 MB/s eta 0:00:00
    torch (>=1.9.*)
           ~~~~~~^
  Attempting uninstall: matplot